# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 12 2023 10:48AM,254905,19,60019240,"GUSA Granules USA, Inc.",Released
1,Jan 12 2023 10:46AM,254903,10,MSP217467,"Methapharm, Inc.",Released
2,Jan 12 2023 10:46AM,254903,10,MSP217468,"Methapharm, Inc.",Released
3,Jan 12 2023 10:46AM,254903,10,MSP217469,"Methapharm, Inc.",Released
4,Jan 12 2023 10:46AM,254903,10,MSP217470,"Methapharm, Inc.",Released
5,Jan 12 2023 10:46AM,254903,10,MSP217471,"Methapharm, Inc.",Released
6,Jan 12 2023 10:46AM,254903,10,MSP217473,"Methapharm, Inc.",Released
7,Jan 12 2023 10:46AM,254903,10,MSP217474,"Methapharm, Inc.",Released
8,Jan 12 2023 10:46AM,254903,10,MSP217475,"Methapharm, Inc.",Released
9,Jan 12 2023 10:46AM,254903,10,MSP217476,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,254900,Released,1
35,254901,Released,4
36,254902,Released,5
37,254903,Released,18
38,254905,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254900,NaN,NaN,1.0
254901,NaN,NaN,4.0
254902,NaN,NaN,5.0
254903,NaN,NaN,18.0
254905,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254770,7.0,3.0,2.0
254806,0.0,0.0,1.0
254810,0.0,1.0,0.0
254811,0.0,1.0,0.0
254813,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254770,7,3,2
254806,0,0,1
254810,0,1,0
254811,0,1,0
254813,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254770,7,3,2
1,254806,0,0,1
2,254810,0,1,0
3,254811,0,1,0
4,254813,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254770,7,3,2
1,254806,,,1
2,254810,,1,
3,254811,,1,
4,254813,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc."
1,Jan 12 2023 10:46AM,254903,10,"Methapharm, Inc."
19,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation
24,Jan 12 2023 10:38AM,254901,10,Methapharm-G
26,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc."
28,Jan 12 2023 10:25AM,254900,21,"OK Capsule, Inc."
29,Jan 12 2023 10:20AM,254899,17,"Citieffe, Inc."
30,Jan 12 2023 9:57AM,254895,10,ISDIN Corporation
47,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation
52,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",,,1
1,Jan 12 2023 10:46AM,254903,10,"Methapharm, Inc.",,,18
2,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation,,,5
3,Jan 12 2023 10:38AM,254901,10,Methapharm-G,,,4
4,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",,,4
5,Jan 12 2023 10:25AM,254900,21,"OK Capsule, Inc.",,,1
6,Jan 12 2023 10:20AM,254899,17,"Citieffe, Inc.",,,1
7,Jan 12 2023 9:57AM,254895,10,ISDIN Corporation,,,17
8,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation,,,5
9,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation,,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",1,,
1,Jan 12 2023 10:46AM,254903,10,"Methapharm, Inc.",18,,
2,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation,5,,
3,Jan 12 2023 10:38AM,254901,10,Methapharm-G,4,,
4,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",4,,
5,Jan 12 2023 10:25AM,254900,21,"OK Capsule, Inc.",1,,
6,Jan 12 2023 10:20AM,254899,17,"Citieffe, Inc.",1,,
7,Jan 12 2023 9:57AM,254895,10,ISDIN Corporation,17,,
8,Jan 12 2023 9:44AM,254894,10,ISDIN Corporation,5,,
9,Jan 12 2023 9:44AM,254893,10,ISDIN Corporation,4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",1,,
1,Jan 12 2023 10:46AM,254903,10,"Methapharm, Inc.",18,,
2,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation,5,,
3,Jan 12 2023 10:38AM,254901,10,Methapharm-G,4,,
4,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,Jan 12 2023 10:46AM,254903,10,"Methapharm, Inc.",18.0,NaN,NaN
2,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation,5.0,NaN,NaN
3,Jan 12 2023 10:38AM,254901,10,Methapharm-G,4.0,NaN,NaN
4,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,Jan 12 2023 10:46AM,254903,10,"Methapharm, Inc.",18.0,0.0,0.0
2,Jan 12 2023 10:40AM,254902,10,ISDIN Corporation,5.0,0.0,0.0
3,Jan 12 2023 10:38AM,254901,10,Methapharm-G,4.0,0.0,0.0
4,Jan 12 2023 10:38AM,254901,10,"Methapharm, Inc.",4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2548820,96.0,4.0,7.0
15,764545,65.0,50.0,33.0
16,509756,1.0,1.0,0.0
17,254899,1.0,0.0,0.0
19,764636,25.0,39.0,0.0
21,3567643,12.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2548820,96.0,4.0,7.0
1,15,764545,65.0,50.0,33.0
2,16,509756,1.0,1.0,0.0
3,17,254899,1.0,0.0,0.0
4,19,764636,25.0,39.0,0.0
5,21,3567643,12.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,96.0,4.0,7.0
1,15,65.0,50.0,33.0
2,16,1.0,1.0,0.0
3,17,1.0,0.0,0.0
4,19,25.0,39.0,0.0
5,21,12.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,96.0
1,15,Released,65.0
2,16,Released,1.0
3,17,Released,1.0
4,19,Released,25.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,19,21
Status,,,,,,
Completed,7.0,33.0,0.0,0.0,0.0,0.0
Executing,4.0,50.0,1.0,0.0,39.0,2.0
Released,96.0,65.0,1.0,1.0,25.0,12.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,19,21
0,Completed,7.0,33.0,0.0,0.0,0.0,0.0
1,Executing,4.0,50.0,1.0,0.0,39.0,2.0
2,Released,96.0,65.0,1.0,1.0,25.0,12.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,19,21
0,Completed,7.0,33.0,0.0,0.0,0.0,0.0
1,Executing,4.0,50.0,1.0,0.0,39.0,2.0
2,Released,96.0,65.0,1.0,1.0,25.0,12.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()